In [1]:
from metrics import *

# Normal results

In [7]:
exp = "normal"
results_path = f'../data/{exp}/results/'

results = pd.read_pickle(results_path + "results_table.pkl")
results

PL            CNN             ABC          CNN_ES
brown  MSE_r    0.4365 (0.60)  0.4804 (0.26)   1.1818 (0.95)   0.5784 (0.49)
       MSE_s    0.0826 (0.08)  0.0502 (0.09)   0.2627 (0.23)   0.0099 (0.01)
       MSE_ext  0.6237 (0.56)  0.2378 (0.22)   0.8710 (1.12)   0.3399 (0.41)
       IS_r                 -              -   4.0076 (0.37)   2.6967 (0.63)
       IS_s                 -              -   1.0973 (0.36)   0.4824 (0.19)
       IIS                  -              -  17.3100 (7.05)  12.1058 (2.66)
       KL                   -              -               -               -
       ES                   -              -   0.7804 (0.22)   0.4479 (0.22)
powexp MSE_r    5.3361 (3.79)  0.6917 (0.67)   1.0755 (1.00)   1.5348 (1.77)
       MSE_s    0.2295 (0.16)  0.0702 (0.07)   0.2672 (0.21)   0.0378 (0.03)
       MSE_ext  0.2272 (0.12)  0.0355 (0.03)   0.0486 (0.03)   0.0345 (0.02)
       IS_r                 -              -   4.2184 (0.02)  8.9426 (11.62)
       IS_s                 -              -   1.9429 (1.08)   0.8371 (0.66)
       IIS                  -              -   4.1858 (1.57)   4.5501 (2.63)
       KL                   -              -               -               -
       ES                   -              -   0.7378 (0.20)   0.6963 (0.54)

In [8]:
print(results.to_latex(index=True,
                  formatters={"name": str.upper},
                  float_format="{:.4f}".format,
))  

\begin{tabular}{llllll}
\toprule
 &  & PL & CNN & ABC & CNN_ES \\
\midrule
\multirow[t]{8}{*}{brown} & MSE_r & 0.4365 (0.60) & 0.4804 (0.26) & 1.1818 (0.95) & 0.5784 (0.49) \\
 & MSE_s & 0.0826 (0.08) & 0.0502 (0.09) & 0.2627 (0.23) & 0.0099 (0.01) \\
 & MSE_ext & 0.6237 (0.56) & 0.2378 (0.22) & 0.8710 (1.12) & 0.3399 (0.41) \\
 & IS_r & - & - & 4.0076 (0.37) & 2.6967 (0.63) \\
 & IS_s & - & - & 1.0973 (0.36) & 0.4824 (0.19) \\
 & IIS & - & - & 17.3100 (7.05) & 12.1058 (2.66) \\
 & KL & - & - & - & - \\
 & ES & - & - & 0.7804 (0.22) & 0.4479 (0.22) \\
\cline{1-6}
\multirow[t]{8}{*}{powexp} & MSE_r & 5.3361 (3.79) & 0.6917 (0.67) & 1.0755 (1.00) & 1.5348 (1.77) \\
 & MSE_s & 0.2295 (0.16) & 0.0702 (0.07) & 0.2672 (0.21) & 0.0378 (0.03) \\
 & MSE_ext & 0.2272 (0.12) & 0.0355 (0.03) & 0.0486 (0.03) & 0.0345 (0.02) \\
 & IS_r & - & - & 4.2184 (0.02) & 8.9426 (11.62) \\
 & IS_s & - & - & 1.9429 (1.08) & 0.8371 (0.66) \\
 & IIS & - & - & 4.1858 (1.57) & 4.5501 (2.63) \\
 & KL & - & - & - & -

# Outside parameters

In [11]:
exp = "outside_parameters"
results_path = f'../data/{exp}/results/'

results = pd.read_pickle(results_path + "results_table.pkl")
results

PL              CNN              ABC  \
brown MSE_r    12.5082 (14.89)  11.0281 (11.03)  18.3440 (16.89)   
      MSE_s      0.1375 (0.22)    0.0683 (0.06)    0.1151 (0.10)   
      MSE_ext    0.4458 (0.36)    1.0855 (1.32)    1.2980 (1.66)   
      IS_r                   -                -  91.4816 (78.15)   
      IS_s                   -                -    1.4500 (0.74)   
      IIS                    -                -  48.5064 (43.59)   
      KL                     -                -                -   
      ES                     -                -    3.0408 (2.16)   

                         CNN_ES  
brown MSE_r     11.3898 (12.64)  
      MSE_s       0.0718 (0.06)  
      MSE_ext     1.1485 (1.24)  
      IS_r      69.7753 (76.12)  
      IS_s        2.5771 (2.58)  
      IIS      96.4928 (106.69)  
      KL                      -  
      ES          2.3646 (2.07)

In [12]:
print(results.to_latex(index=True,
                  formatters={"name": str.upper},
                  float_format="{:.4f}".format,
))  

\begin{tabular}{llllll}
\toprule
 &  & PL & CNN & ABC & CNN_ES \\
\midrule
\multirow[t]{8}{*}{brown} & MSE_r & 12.5082 (14.89) & 11.0281 (11.03) & 18.3440 (16.89) & 11.3898 (12.64) \\
 & MSE_s & 0.1375 (0.22) & 0.0683 (0.06) & 0.1151 (0.10) & 0.0718 (0.06) \\
 & MSE_ext & 0.4458 (0.36) & 1.0855 (1.32) & 1.2980 (1.66) & 1.1485 (1.24) \\
 & IS_r & - & - & 91.4816 (78.15) & 69.7753 (76.12) \\
 & IS_s & - & - & 1.4500 (0.74) & 2.5771 (2.58) \\
 & IIS & - & - & 48.5064 (43.59) & 96.4928 (106.69) \\
 & KL & - & - & - & - \\
 & ES & - & - & 3.0408 (2.16) & 2.3646 (2.07) \\
\cline{1-6}
\bottomrule
\end{tabular}



# Outside model

## Whitmat scenario

In [13]:
exp = "outside_model"
results_path = f'../data/{exp}/results/'

results = pd.read_pickle(results_path + "whitmat_results_table.pkl")
results

PL            CNN            ABC           CNN_ES
whitmat MSE_r    9.4912 (3.01)  1.1578 (0.55)  0.2973 (0.30)    4.6697 (2.84)
        MSE_s    0.6587 (0.75)  0.1638 (0.23)  0.0843 (0.10)    0.3001 (0.25)
        MSE_ext  0.7415 (0.22)  0.0324 (0.04)  0.1406 (0.11)    0.0090 (0.01)
        IS_r                 -              -  4.2761 (0.03)  16.6463 (13.79)
        IS_s                 -              -  1.4050 (0.01)    9.2161 (7.61)
        IIS                  -              -  9.0895 (9.59)    1.9807 (0.38)
        KL                   -              -              -                -
        ES                   -              -  0.5088 (0.10)    1.5447 (0.51)

In [14]:
print(results.to_latex(index=True,
                  formatters={"name": str.upper},
                  float_format="{:.4f}".format,
))  

\begin{tabular}{llllll}
\toprule
 &  & PL & CNN & ABC & CNN_ES \\
\midrule
\multirow[t]{8}{*}{whitmat} & MSE_r & 9.4912 (3.01) & 1.1578 (0.55) & 0.2973 (0.30) & 4.6697 (2.84) \\
 & MSE_s & 0.6587 (0.75) & 0.1638 (0.23) & 0.0843 (0.10) & 0.3001 (0.25) \\
 & MSE_ext & 0.7415 (0.22) & 0.0324 (0.04) & 0.1406 (0.11) & 0.0090 (0.01) \\
 & IS_r & - & - & 4.2761 (0.03) & 16.6463 (13.79) \\
 & IS_s & - & - & 1.4050 (0.01) & 9.2161 (7.61) \\
 & IIS & - & - & 9.0895 (9.59) & 1.9807 (0.38) \\
 & KL & - & - & - & - \\
 & ES & - & - & 0.5088 (0.10) & 1.5447 (0.51) \\
\cline{1-6}
\bottomrule
\end{tabular}



## Smith scenario

In [15]:
exp = "outside_model"
results_path = f'../data/{exp}/results/'

results = pd.read_pickle(results_path + "brown_results_table.pkl")
results

PL            CNN                ABC         CNN_ES
brown MSE_r    1.6732 (1.16)  2.1668 (1.77)      2.6969 (1.83)  1.1888 (0.99)
      MSE_s    0.0949 (0.16)  0.2256 (0.11)      1.1173 (0.79)  0.0314 (0.02)
      MSE_ext  0.2863 (0.18)  0.2090 (0.12)      2.3387 (2.36)  0.1425 (0.12)
      IS_r                 -              -      4.6666 (1.33)  3.9836 (2.92)
      IS_s                 -              -    13.8325 (19.15)  2.1269 (1.04)
      IIS                  -              -  123.3102 (150.78)  7.0149 (5.11)
      KL                   -              -                  -              -
      ES                   -              -      1.3561 (0.40)  0.7028 (0.45)

In [16]:
print(results.to_latex(index=True,
                  formatters={"name": str.upper},
                  float_format="{:.4f}".format,
))  

\begin{tabular}{llllll}
\toprule
 &  & PL & CNN & ABC & CNN_ES \\
\midrule
\multirow[t]{8}{*}{brown} & MSE_r & 1.6732 (1.16) & 2.1668 (1.77) & 2.6969 (1.83) & 1.1888 (0.99) \\
 & MSE_s & 0.0949 (0.16) & 0.2256 (0.11) & 1.1173 (0.79) & 0.0314 (0.02) \\
 & MSE_ext & 0.2863 (0.18) & 0.2090 (0.12) & 2.3387 (2.36) & 0.1425 (0.12) \\
 & IS_r & - & - & 4.6666 (1.33) & 3.9836 (2.92) \\
 & IS_s & - & - & 13.8325 (19.15) & 2.1269 (1.04) \\
 & IIS & - & - & 123.3102 (150.78) & 7.0149 (5.11) \\
 & KL & - & - & - & - \\
 & ES & - & - & 1.3561 (0.40) & 0.7028 (0.45) \\
\cline{1-6}
\bottomrule
\end{tabular}

